# Semantic Kernel s integrací OpenBnB MCP serveru

Tento notebook ukazuje, jak používat Semantic Kernel s reálným OpenBnB MCP serverem k vyhledávání skutečných ubytování Airbnb pomocí MCPStdioPlugin. Pro přístup k LLM využívá Azure AI Foundry. Pro nastavení vašich environmentálních proměnných můžete postupovat podle [Setup Lesson](/00-course-setup/README.md)


## Importujte potřebné balíčky


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Vytvoření připojení MCP pluginu

Připojíme se k [OpenBnB MCP serveru](https://github.com/openbnb-org/mcp-server-airbnb) pomocí MCPStdioPlugin. Tento server poskytuje funkci vyhledávání Airbnb prostřednictvím balíčku @openbnb/mcp-server-airbnb.


## Vytvoření klienta

V tomto příkladu použijeme Azure AI Foundry pro přístup k LLM. Ujistěte se, že vaše proměnné prostředí jsou správně nastaveny.


## Konfigurace prostředí

Nastavte parametry Azure OpenAI. Ujistěte se, že máte nastavené následující proměnné prostředí:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Porozumění integraci OpenBnB MCP

Tento notebook se připojuje k **reálnému OpenBnB MCP serveru**, který poskytuje skutečnou funkčnost vyhledávání na Airbnb.

### Jak to funguje:

1. **MCPStdioPlugin**: Používá komunikaci přes standardní vstup/výstup s MCP serverem
2. **Reálný NPM balíček**: Stáhne a spustí `@openbnb/mcp-server-airbnb` pomocí npx
3. **Živá data**: Vrací skutečná data o nemovitostech z Airbnb prostřednictvím jejich API
4. **Objevování funkcí**: Agent automaticky objevuje dostupné funkce z MCP serveru

### Dostupné funkce:

OpenBnB MCP server obvykle poskytuje:
- **search_listings** - Vyhledávání nemovitostí na Airbnb podle lokality a kritérií
- **get_listing_details** - Získání podrobných informací o konkrétních nemovitostech
- **check_availability** - Kontrola dostupnosti pro konkrétní data
- **get_reviews** - Získání recenzí na nemovitosti
- **get_host_info** - Získání informací o hostitelích nemovitostí

### Požadavky:

- **Node.js** nainstalovaný ve vašem systému
- **Připojení k internetu** pro stažení balíčku MCP serveru
- **NPX** k dispozici (součástí Node.js)

### Testování připojení:

Můžete ručně otestovat MCP server spuštěním:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Tím se stáhne a spustí OpenBnB MCP server, ke kterému se Semantic Kernel připojí pro získání reálných dat z Airbnb.


## Spuštění agenta s OpenBnB MCP serverem

Nyní spustíme AI agenta, který se připojuje k OpenBnB MCP serveru, aby vyhledal skutečné ubytování Airbnb ve Stockholmu pro 2 dospělé a 1 dítě. Klidně upravte seznam `user_inputs`, pokud chcete změnit kritéria vyhledávání.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Shrnutí
Gratulujeme! Úspěšně jste vytvořili AI agenta, který se integruje s reálným vyhledáváním ubytování pomocí Model Context Protocol (MCP):

## Použité technologie:
- Semantic Kernel - Pro vytváření inteligentních agentů s Azure OpenAI
- Azure AI Foundry - Pro schopnosti LLM a dokončování chatů
- MCP (Model Context Protocol) - Pro standardizovanou integraci nástrojů
- OpenBnB MCP Server - Pro skutečnou funkčnost vyhledávání na Airbnb
- Node.js/NPX - Pro spuštění externího MCP serveru

## Co jste se naučili:
- Integrace MCP: Propojení agentů Semantic Kernel s externími MCP servery
- Přístup k datům v reálném čase: Vyhledávání skutečných nemovitostí na Airbnb prostřednictvím živých API
- Komunikace protokolu: Použití stdio komunikace mezi agentem a MCP serverem
- Objevování funkcí: Automatické objevování dostupných funkcí z MCP serverů
- Streamování odpovědí: Zachycování a logování volání funkcí v reálném čase
- HTML vykreslování: Formátování odpovědí agenta pomocí stylizovaných tabulek a interaktivních zobrazení

## Další kroky:
- Integrace dalších MCP serverů (počasí, lety, restaurace)
- Vytvoření systému s více agenty kombinujícího MCP a A2A protokoly
- Vytvoření vlastních MCP serverů pro vaše datové zdroje
- Implementace trvalé paměti konverzací napříč relacemi
- Nasazení agenta do Azure Functions s orchestrací MCP serveru
- Přidání uživatelské autentizace a možností rezervace



---

**Upozornění**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o co největší přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Za závazný zdroj by měl být považován původní dokument v jeho původním jazyce. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné výklady vyplývající z použití tohoto překladu.
